In [1]:
import pandas as pd
import numpy as np
from scipy import spatial
import operator

In [2]:
def rating_mat(file_name):
        df = pd.read_csv(file_name)
        df_pivot = df.pivot(index='userId', columns='movieId', values='rating')
        mat = df_pivot.values
        return mat

In [3]:
mat = rating_mat('movie_ratings.csv')

In [4]:
def dataSplit(mat, split = 0.9):
        n = split
        all_indices = np.argwhere(~np.isnan(mat))
        np.random.shuffle(all_indices)
        train_ind, test_ind = all_indices[:int(n*len(all_indices))], all_indices[int(n*len(all_indices))+1:]
        train = np.empty(mat.shape)
        test = np.empty(mat.shape)
        train[:] = np.nan
        test[:] = np.nan
        for i in train_ind:
            train[i[0], i[1]] = mat[i[0], i[1]]
        for j in test_ind:
            test[j[0], j[1]] = mat[j[0], j[1]]
        
        return train, test

In [5]:
train, test = dataSplit(mat)

In [6]:
train

array([[4. , nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [2.5, 2. , 2. , ..., nan, nan, nan],
       [3. , nan, nan, ..., nan, nan, nan],
       [5. , nan, nan, ..., nan, nan, nan]])

In [7]:
test

array([[nan, nan,  4., ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [8]:
def ComputeSimilarity(a, b): ## a = arr([0.0, 3.0, nan...])
    q = np.argwhere(~np.isnan(a))
    q = set([x[0] for x in q])
    
    r = np.argwhere(~np.isnan(b))
    r = set([y[0] for y in r])
    t = r & q
    t = list(t)
    if len(t) > 0:
        userDistance = 1-spatial.distance.cosine(a[t], b[t])
        return userDistance
    else:
        return 0

In [9]:
def getNeighbors(userID, K):
    distances = []
    for user in userDict:
        if (user != userID):
            dist = ComputeSimilarity(userDict[user], userDict[userID])
            distances.append((user, dist))
    distances.sort(key=operator.itemgetter(1), reverse = True)
    neighbors = []
    for x in range(K):
        neighbors.append(distances[x][0])
    return neighbors

In [10]:
userID = np.array(range(len(train)+1))

In [11]:
userID = userID[1:]

In [12]:
userID

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

In [13]:
userDict = {}
for i in range(len(train)):
    userDict[i+1] = train[i]

In [14]:
userDict

{1: array([ 4., nan, nan, ..., nan, nan, nan]),
 2: array([nan, nan, nan, ..., nan, nan, nan]),
 3: array([nan, nan, nan, ..., nan, nan, nan]),
 4: array([nan, nan, nan, ..., nan, nan, nan]),
 5: array([ 4., nan, nan, ..., nan, nan, nan]),
 6: array([nan,  4.,  5., ..., nan, nan, nan]),
 7: array([4.5, nan, nan, ..., nan, nan, nan]),
 8: array([nan,  4., nan, ..., nan, nan, nan]),
 9: array([nan, nan, nan, ..., nan, nan, nan]),
 10: array([nan, nan, nan, ..., nan, nan, nan]),
 11: array([nan, nan, nan, ..., nan, nan, nan]),
 12: array([nan, nan, nan, ..., nan, nan, nan]),
 13: array([nan, nan, nan, ..., nan, nan, nan]),
 14: array([nan, nan, nan, ..., nan, nan, nan]),
 15: array([2.5, nan, nan, ..., nan, nan, nan]),
 16: array([nan, nan, nan, ..., nan, nan, nan]),
 17: array([4.5, nan, nan, ..., nan, nan, nan]),
 18: array([3.5, 3. , nan, ..., nan, nan, nan]),
 19: array([ 4., nan,  3., ..., nan, nan, nan]),
 20: array([nan,  3., nan, ..., nan, nan, nan]),
 21: array([3.5, 3.5, nan, ..

In [15]:
def meanRating(i):
    return np.nanmean(i)

In [16]:
def predictRating(test_user, NN, j):   ## test_user = arr()
    mean_test_user = meanRating(test_user)
    
    dist_NN = 0
    for i in NN:
        dist_NN += ComputeSimilarity(test_user, userDict[i])



#         if np.isnan(z):
#             dist_NN +=1000
#         else:
#             dist_NN +=z
    
    if dist_NN == 0:
        K = 0
    else:
        K = 1 / dist_NN
    temp = 0
    for l in NN:
        a = (ComputeSimilarity(test_user, userDict[l]))*(userDict[l][j] - meanRating(userDict[l]))
        if np.isnan(a):
            temp+=0
        else:
            temp+=a

        
    prediction = mean_test_user + K*temp
    
    return prediction

In [17]:
test_userDict = {}
for i in range(len(test)):
    test_userDict[i+1] = test[i]

In [18]:
test_userDict

{1: array([nan, nan,  4., ..., nan, nan, nan]),
 2: array([nan, nan, nan, ..., nan, nan, nan]),
 3: array([nan, nan, nan, ..., nan, nan, nan]),
 4: array([nan, nan, nan, ..., nan, nan, nan]),
 5: array([nan, nan, nan, ..., nan, nan, nan]),
 6: array([nan, nan, nan, ..., nan, nan, nan]),
 7: array([nan, nan, nan, ..., nan, nan, nan]),
 8: array([nan, nan, nan, ..., nan, nan, nan]),
 9: array([nan, nan, nan, ..., nan, nan, nan]),
 10: array([nan, nan, nan, ..., nan, nan, nan]),
 11: array([nan, nan, nan, ..., nan, nan, nan]),
 12: array([nan, nan, nan, ..., nan, nan, nan]),
 13: array([nan, nan, nan, ..., nan, nan, nan]),
 14: array([nan, nan, nan, ..., nan, nan, nan]),
 15: array([nan, nan, nan, ..., nan, nan, nan]),
 16: array([nan, nan, nan, ..., nan, nan, nan]),
 17: array([nan, nan, nan, ..., nan, nan, nan]),
 18: array([nan, nan, nan, ..., nan, nan, nan]),
 19: array([nan,  3., nan, ..., nan, nan, nan]),
 20: array([nan, nan, nan, ..., nan, nan, nan]),
 21: array([nan, nan, nan, ..

In [19]:
test_ind = np.argwhere(~np.isnan(test))
SE = 0
pred_ij = np.empty(test.shape)
for user in test_userDict:
    NN = getNeighbors(user, 5)
    test_arr = np.argwhere(~np.isnan(test_userDict[user]))
    for j in test_arr:
        pred_ij[user-1, j] = predictRating(test_userDict[user], NN, j)
#     for item in test_ind:
#         pred_ij = predictRating(test_userDict[user], NN, item[1])
#         error = (test[item[0],item[1]] - pred_ij)**2
#         SE += error

# MSE = (SE/len(test_ind))

In [20]:
for i in test_ind:
    error = (test[i[0], i[1]] - pred_ij[i[0], i[1]])**2
    SE += error
MSE = SE / len(test_ind)    
print(MSE)

0.8330560803462177


In [21]:
test_ind = np.argwhere(~np.isnan(test))
SE = 0
pred_ij = np.empty(test.shape)
for user in userDict:
    NN = getNeighbors(user, 5)
    train_arr = np.argwhere(~np.isnan(userDict[user]))
    for j in train_arr:
        pred_ij[user-1, j] = predictRating(userDict[user], NN, j)

In [22]:
train_ind = np.argwhere(~np.isnan(train))
for i in train_ind:
    error = (train[i[0], i[1]] - pred_ij[i[0], i[1]])**2
    SE += error
MSE_train = SE / len(train_ind)    
print(MSE_train)

0.8728696527885851


In [23]:
def KNN(file_name, k):
    def rating_mat(file_name):
        df = pd.read_csv(file_name)
        df_pivot = df.pivot(index='userId', columns='movieId', values='rating')
        mat = df_pivot.values
        return mat
    
    mat = rating_mat(file_name)
    
    def dataSplit(mat, split = k):
        n = split
        all_indices = np.argwhere(~np.isnan(mat))
        np.random.shuffle(all_indices)
        train_ind, test_ind = all_indices[:int(n*len(all_indices))], all_indices[int(n*len(all_indices))+1:]
        train = np.empty(mat.shape)
        test = np.empty(mat.shape)
        train[:] = np.nan
        test[:] = np.nan
        for i in train_ind:
            train[i[0], i[1]] = mat[i[0], i[1]]
        for j in test_ind:
            test[j[0], j[1]] = mat[j[0], j[1]]
        
        return train, test
    
    train, test = dataSplit(mat)
    userID = np.array(range(len(train)+1))
    userID = userID[1:]
    userDict = {}
    for i in range(len(train)):
        userDict[i+1] = train[i]

    test_userDict = {}
    for i in range(len(test)):
        test_userDict[i+1] = test[i]

    test_ind = np.argwhere(~np.isnan(test))
    
    def ComputeSimilarity(a, b): 
        q = np.argwhere(~np.isnan(a))
        q = set([x[0] for x in q])
    
        r = np.argwhere(~np.isnan(b))
        r = set([y[0] for y in r])
        t = r & q
        t = list(t)
        if len(t) > 0:
            userDistance = 1-spatial.distance.cosine(a[t], b[t])
            return userDistance
        else:
            return 0
    
    def getNeighbors(userID, K):
        distances = []
        for user in userDict:
            if (user != userID):
                dist = ComputeSimilarity(userDict[user], userDict[userID])
                distances.append((user, dist))
        distances.sort(key=operator.itemgetter(1), reverse = True)
        neighbors = []
        for x in range(K):
            neighbors.append(distances[x][0])
        return neighbors
    
    def meanRating(i):
        return np.nanmean(i)
    
    def predictRating(test_user, NN, j):
        mean_test_user = meanRating(test_user)
    
        dist_NN = 0
        for i in NN:
            dist_NN += ComputeSimilarity(test_user, userDict[i])
 
        if dist_NN == 0:
            K = 0
        else:
            K = 1 / dist_NN
        temp = 0
        for l in NN:
            a = (ComputeSimilarity(test_user, userDict[l]))*(userDict[l][j] - meanRating(userDict[l]))
            if np.isnan(a):
                temp+=0
            else:
                temp+=a
        prediction = mean_test_user + K*temp
    
        return prediction
    
    def PredictionMatrix(test, test_userDict):
        SE = 0
        pred_ij = np.empty(test.shape)
        for user in test_userDict:
            NN = getNeighbors(user, 5)
            test_arr = np.argwhere(~np.isnan(test_userDict[user]))
            for j in test_arr:
                pred_ij[user-1, j] = predictRating(test_userDict[user], NN, j)
        return pred_ij
        for i in test_ind:
            error = (test[i[0], i[1]] - pred_ij[i[0], i[1]])**2
            SE += error
        MSE = SE / len(test_ind)    
        print('The MSE for split = ', k, ' is ',  MSE)
    
    PredictionMatrix(test, test_userDict)

In [25]:
file_name = 'movie_ratings.csv'
mat = rating_mat(file_name)
train, test = dataSplit(mat)
userID = np.array(range(len(train)+1))
userID = userID[1:]

In [26]:
userDict = {}
for i in range(len(train)):
    userDict[i+1] = train[i]

test_userDict = {}
for i in range(len(test)):
    test_userDict[i+1] = test[i]

test_ind = np.argwhere(~np.isnan(test))

In [27]:
def PredictionMatrix(test, test_userDict, test_ind):
    SE = 0
    pred_ij = np.empty(test.shape)
    for user in test_userDict:
        NN = getNeighbors(user, 5)
        test_arr = np.argwhere(~np.isnan(test_userDict[user]))
        for j in test_arr:
            pred_ij[user-1, j] = predictRating(test_userDict[user], NN, j)
    for i in test_ind:
        error = (test[i[0], i[1]] - pred_ij[i[0], i[1]])**2
        SE += error
    MSE = SE / len(test_ind)    
    print(MSE)




In [29]:
pred_ij = PredictionMatrix(test, test_userDict, test_ind)
for i in test_ind:
    error = (test[i[0], i[1]] - pred_ij[i[0], i[1]])**2
    SE += error
MSE = SE / len(test_ind)    
print(MSE)

0.826823635073341


TypeError: 'NoneType' object is not subscriptable

In [ ]:
for i in range(2, 9):
    k = 0.1*i
    KNN('movie_ratings.csv', k)